# Shifted Beta-Geometric Modeling with Cohorts

In this notebook we replicate the main results and figures from ["How to Project Customer Retention"](https://faculty.wharton.upenn.edu/wp-content/uploads/2012/04/Fader_hardie_jim_07.pdf) by Hardie & Fader (2007), which introduces the Shifted Beta-Geometric (sBG) model for customer behavior in a discrete contractual setting. It is ideal for business cases involving recurring subscriptions and has the following assumptions:
* Customer cancellation probabilities are Beta-distributed with hyperparameters `alpha` and `beta`.
* Retention rates change over time due to customer heterogeneity.
* All customers in a given cohort began their contract in the same time period.

The last assumption in particular is an ideal application for hierarchical Bayesian modeling.

# Tutorial Notebook Outstanding Tasks
- Merge PR for `expected_retention_elasticity` and `expected_lifetime_purchases` predictive methods
- Merge Static Covariate PR
- Merge Mean/Polarization index plotting PR
- Fit regular/highend data to both a cohort and covariate model to replicate research results
- Synthesize time cohort model with covariates to showcase conventional cohort EDA, additional predictive methods, and mean/polar plotting

In [1]:
import pytensor

#set flag to hotfix open c++ errors
pytensor.config.cxx = '/usr/bin/clang++'

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from pymc_marketing import clv
import pymc as pm
from pymc_extras.prior import Prior

# Plotting configuration
az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

# Load Data and EDA
Adapt plots from sBG-Individual notebook here.

In [3]:
dataset = pd.read_csv("https://raw.githubusercontent.com/ColtAllen/pymc-marketing/refs/heads/clv-sbg-model/data/sbg_reg_hi_cohorts.csv")

In [185]:
# Data from research paper
df = pd.DataFrame(
    {
        "regular": [
            100.0,
            63.1,
            46.8,
            38.2,
            32.6,
            28.9,
            26.2,
            24.1,
            22.3,
            20.7,
            19.4,
            18.3,
            17.3,
        ],
        "highend": [
            100.0,
            86.9,
            74.3,
            65.3,
            59.3,
            55.1,
            51.7,
            49.1,
            46.8,
            44.5,
            42.7,
            40.9,
            39.4,
        ],
    }
)
df

,regular,highend
0,100.0,100.0
1,63.1,86.9
2,46.8,74.3
3,38.2,65.3
4,32.6,59.3
5,28.9,55.1
6,26.2,51.7
7,24.1,49.1
8,22.3,46.8
9,20.7,44.5


Figures 4 and 5 show the predicted average churn and retention trends for the two groups.
We can see that predictions nicely match the observed data (black line), even when extrapolating into the time periods that were held-out when fitting the model.

The plots also highlight an interesting implication from the model: 
the retention rates are expected to increase over time, as the more precarious customers gradually drop out. This is a direct consequence of modelling individual churn rates as being fixed over time.

## Model Fitting
#### High-end and Regular User Cohorts

In [7]:
sbg = clv.ShiftedBetaGeoModel(
    data=dataset,
    model_config = {
        "alpha": Prior("HalfFlat",dims="cohort"),
        "beta": Prior("HalfFlat",dims="cohort")
    }
)
sbg.build_model()
sbg.fit(fit_method='map')
sbg.fit_summary()

Output()

alpha[highend]    0.668
alpha[regular]    0.704
beta[highend]     3.806
beta[regular]     1.182
Name: value, dtype: float64

In [4]:
sbg = clv.ShiftedBetaGeoModel(
    data=dataset,
)
sbg.build_model()
sbg.fit(fit_method='mcmc')

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [phi, kappa]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 7 seconds.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> fit_data

## Contrasting posterior inferences with the repo MLE estimates

The sBG model has 2 population parameters of interest: `alpha` and `beta`. 
These parameters define the population distribution of the latent churn rate distribution `theta`. 
The larger the values of `alpha` and `beta`, the more homogenous the churn rates across different customers. 

The ratio of `alpha` to `beta` tells us the expected churn rates. If `alpha/beta == 0.1`, we expect the average customer to have a `0.1` probability of churning between each time period.

The model fitting agrees with the Maximum Likelihood estimates described in the original paper.
In addition, MCMC sampling, gives us useful information about the uncertainty of the fits.

In [5]:
# MLE estimates from the paper
ref_val = {
    "highend": [0.668, 3.806],
    "regular": [0.704, 1.182],
}
sbg.fit_summary()
#az.plot_trace(sbg.idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
phi[highend],0.150,0.009,0.133,0.168,0.000,0.000,2359.0,2204.0,1.0
phi[regular],0.374,0.015,0.347,0.402,0.000,0.000,2820.0,3061.0,1.0
kappa[highend],4.518,0.947,3.016,6.277,0.019,0.024,2512.0,2873.0,1.0
kappa[regular],1.880,0.208,1.512,2.278,0.004,0.003,2625.0,2599.0,1.0
alpha[highend],0.672,0.110,0.486,0.871,0.002,0.002,3025.0,2972.0,1.0
alpha[regular],0.702,0.064,0.584,0.819,0.001,0.001,3256.0,2897.0,1.0
beta[highend],3.846,0.842,2.510,5.397,0.017,0.022,2469.0,2769.0,1.0
beta[regular],1.178,0.150,0.914,1.459,0.003,0.003,2496.0,2495.0,1.0


# Predictive Methods
Adapt plotting code from sBG-Individual notebook here.

In [ ]:
az.plot_hdi(
    weeks_,
    hi_retention.mean("customer_id"),
    hdi_prob=0.95,
    color="C0",
    fill_kwargs={"label": "high end"},
)
az.plot_hdi(
    weeks_,
    lo_retention.mean("customer_id"),
    hdi_prob=0.95,
    color="C1",
    fill_kwargs={"label": "regular"},
)

plt.plot(weeks_, retention_rate_highend_obs, color="k", label="observed")
plt.plot(weeks_, retention_rate_regular_obs, color="k")

plt.axvline(7, ls="--", color="k")
plt.ylim([0.5, 1.05])
plt.ylabel("Retention Rate")
plt.legend()
plt.title("Figure 5");

In [ ]:
obs = df["highend"]/100

plt.plot(survive_pred,color="b", label="predicted")
plt.plot(obs,color="k", label="observed")
plt.ylabel("Survival Rate")
plt.legend()
plt.title("High-End Customers MAP")

plt.plot(reg_retention_cohort[:8],color="b", label="predicted")
plt.plot(retention_rate_regular_obs[:8],color="k", label="observed")
plt.ylabel("Retention Rate")
plt.legend()
plt.title("Regular Customers MCMC Estimated with Cohorts")

In [120]:
pred_data=dataset.query("recency==T")
pred_cohort_retention = sbg.expected_retention_rate(pred_data,future_t=0).mean(("chain","draw"))

pred_cohort_retention.to_dataframe(name="retention").reset_index()

,cohort,customer_id,retention
0,highend,510,0.943356
1,highend,511,0.943356
2,highend,512,0.943356
3,highend,513,0.943356
4,highend,514,0.943356
...,...,...,...
727,regular,1996,0.921894
728,regular,1997,0.921894
729,regular,1998,0.921894
730,regular,1999,0.921894


In [34]:
max_T = 12
cohort_names = np.array(["regular","highend"])
cohorts_covar = np.array([0,1])
T_rng = np.arange(1,max_T+1,1)

pred_data = pd.DataFrame(
    {
        "customer_id": np.arange(1,1+max_T*2,1),
        "T": np.repeat(T_rng,len(cohort_names)),
        "cohort": np.tile(cohort_names,max_T),
        "covar_cohort": np.tile(cohorts_covar,max_T),
    }
)

,customer_id,T,cohort,covar_cohort
0,1,1,regular,0
1,2,1,highend,1
2,3,2,regular,0
3,4,2,highend,1
4,5,3,regular,0
5,6,3,highend,1
6,7,4,regular,0
7,8,4,highend,1
8,9,5,regular,0
9,10,5,highend,1


In [38]:
max_T = 12
cohort_names = np.array(["regular","highend"])
cohorts_covar = np.array([0,1])
T_rng = np.arange(1,max_T+1,1)


pred_data = pd.DataFrame(
    {
        "customer_id": np.arange(1,1+max_T*2,1),
        "T": np.repeat(T_rng,len(cohort_names)),
        "cohort": np.tile(cohort_names,max_T),
        "covar_cohort": np.tile(cohorts_covar,max_T),
    }
)

sbg.expected_retention_rate(
            data=pred_data,
            future_t=0,
        )

# expected_purchases_over_time = xr.concat(
#     objs=[
#         sbg.expected_retention_rate(
#             data=time_data,
#             future_t=0,
#         )
#         for t in range(time_periods)
#     ],
#     dim="t",
# ).transpose(..., "t")

# unique_cohorts = np.unique(expected_purchases_over_time.coords["cohort"].values)

# for cohort in unique_cohorts:
#     print(expected_purchases_over_time.sel(cohort=cohort).mean("t"))

TypeError: ShiftedBetaGeoModel._validate_cohorts() got an unexpected keyword argument 'check_names'

# Additional Predictive Methods

Below methods are described in https://faculty.wharton.upenn.edu/wp-content/uploads/2012/04/Fader_hardie_contractual_mksc_10.pdf, and will be added in a future PR.

### Simulate Data from Follow-Up Paper
It is not expected this dataset will be required, but code to recreate is provided below.

In [14]:
import numpy as np
import pandas as pd

# Cohort table from the markdown (empty cells omitted)
# only need the params for unit testing
# alpha 3.80, beta = 15.20
cohort_counts = {
    2001: {2001: 10000, 2002: 8000, 2003: 6480, 2004: 5307, 2005: 4391},
    2002: {2002: 10000, 2003: 8000, 2004: 6480, 2005: 5307},
    2003: {2003: 10000, 2004: 8000, 2005: 6480},
    2004: {2004: 10000, 2005: 8000},
    2005: {2005: 10000},
}

# alpha=0.067, beta=0.267
case2 = {
    2003: {2003: 10000, 2004: 8000, 2005: 7600, 2006: 7383, 2007: 7235},
    2004: {2004: 10000, 2005: 8000, 2006: 7600, 2007: 7383},
    2005: {2005: 10000, 2006: 8000, 2007: 7600},
    2006: {2006: 10000, 2007: 8000},
    2007: {2007: 10000},
}
# (Bottom totals row is intentionally ignored)

rows = []
for cohort, year_counts in cohort_counts.items():
    years_sorted = sorted(year_counts)
    ages = [y - cohort for y in years_sorted]
    S = [year_counts[y] for y in years_sorted]  # survivors per age
    last_t = ages[-1]  # observation window for this cohort (right-censoring time T)

    # Churn in interval t -> t+1 (event observed at t)
    for t in range(len(S) - 1):
        count = S[t] - S[t + 1]
        if count > 0:
            rows.append({"cohort": cohort, "t_churn": t, "T": last_t, "count": count})

    # Right-censored at last observed age (still active)
    if S[-1] > 0:
        rows.append({"cohort": cohort, "t_churn": last_t, "T": last_t, "count": S[-1]})

counts_df = pd.DataFrame(rows)

# Expand to one row per customer
t_churn_array = np.repeat(counts_df["t_churn"].to_numpy(), counts_df["count"].to_numpy())
T_array = np.repeat(counts_df["T"].to_numpy(), counts_df["count"].to_numpy())
cohort_array = np.repeat(counts_df["cohort"].to_numpy(), counts_df["count"].to_numpy())
customer_id = np.arange(t_churn_array.size)

df = pd.DataFrame(
    {
        "customer_id": customer_id + 1,
        "recency": t_churn_array + 1,
        "T": T_array + 1,
        "cohort": cohort_array,
    }
).astype({"customer_id": int, "t_churn": int, "T": int, "cohort": str})

#df.to_csv("sbg_cohorts.csv",index=False)

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 't_churn' not found in columns."

##### Expected Retention Elasticity

In [74]:
    def expected_retention_elasticity(
        self,
        data: pd.DataFrame | None = None,
        *,
        discount_rate: float = 0.0,
    ) -> xarray.DataArray:
        """Compute expected retention elasticity.

        Adapted from equation (8) in [1]_.

        References
        ----------
        .. [1] Fader, P. S., & Hardie, B. G. (2010). Customer-Base Valuation in a Contractual Setting:
               The Perils of Ignoring Heterogeneity. Marketing Science, 29(1), 85-93.
               https://faculty.wharton.upenn.edu/wp-content/uploads/2012/04/Fader_hardie_contractual_mksc_10.pdf
        """
        if data is None:
            data = self.data

        dataset = self._extract_predictive_variables(
            data, customer_varnames=["recency", "T"]
        )
        alpha = dataset["alpha"]
        beta = dataset["beta"]
        T = dataset["T"]

        retention_elasticity = hyp2f1(
            1, beta + T - 1, alpha + beta + 1, 1 / (1 + discount_rate)
        )
        # TODO: "cohorts" dim instead of "customer_id"?
        return retention_elasticity.transpose(
            "chain", "draw", "cohorts", "customer_id", missing_dims="ignore"
        )

AttributeError: 'ShiftedBetaGeoModel' object has no attribute 'expected_lifetime_purchases'

##### Discounted Expected Lifetime Renewals

In [75]:
    def expected_lifetime_purchases(
        self,
        data: pd.DataFrame | None = None,
        *,
        discount_rate: float = 0.0,
    ) -> xarray.DataArray:
        """Compute expected lifetime purchases.

        Adapted from equation (6) in [1]_.

        References
        ----------
        .. [1] Fader, P. S., & Hardie, B. G. (2010). Customer-Base Valuation in a Contractual Setting:
               The Perils of Ignoring Heterogeneity. Marketing Science, 29(1), 85-93.
               https://faculty.wharton.upenn.edu/wp-content/uploads/2012/04/Fader_hardie_contractual_mksc_10.pdf
        """
        if data is None:
            data = self.data

        dataset = self._extract_predictive_variables(
            data, customer_varnames=["recency", "T"]
        )
        alpha = dataset["alpha"]
        beta = dataset["beta"]
        T = dataset["T"]

        retention_rate = (beta + T - 1) / (alpha + beta + T - 1)
        retention_elasticity = hyp2f1(
            1, beta + T, alpha + beta, 1 / (1 + discount_rate)
        )
        expected_lifetime_purchases = retention_rate * retention_elasticity
        # TODO: "cohorts" dim instead of "customer_id"?
        return expected_lifetime_purchases.transpose(
            "chain", "draw", "customer_id", missing_dims="ignore"
        )

AttributeError: 'ShiftedBetaGeoModel' object has no attribute 'expected_retention_elasticity'

### Latent Dropout Distribution for Customer Population
Distribution mean and polarization indices may be more pertinent to add in a future PR rather than visualizations of the marginalized posterior dropout distributions per cohort, but code to do so is provided below:

In [ ]:
alpha_fit = sbg.fit_result["alpha"]
beta_fit = sbg.fit_result["beta"]

rng = np.random.default_rng(42)
theta_sim = rng.beta(alpha_fit,beta_fit)

theta = xr.DataArray(
    theta_sim,
    dims=("chains","draws","cohort"),
    coords={
        "chains": np.arange(4), 
        "draws": np.arange(1000),
        "cohort": ["highend","regular"],
    },
    name="theta",
)

az_theta = az.convert_to_inference_data(theta)

highend = az_theta.sel(cohort="highend")
highend
highend.posterior["theta"].values.flatten()

az.plot_dist(highend.posterior["theta"].values.flatten())

az_theta.mean(("chains","draws"))

# plot this by cohort
theta
az.plot_dist(theta)

In [ ]:
# Map cohort-wise [alpha, beta] refs to ArviZ variable labels
ref_val_map = {}
for cohort, (a_ref, b_ref) in ref_val.items():
    ref_val_map[f"alpha\n{cohort}"] = a_ref
    ref_val_map[f"beta\n{cohort}"]  = b_ref
   
az.plot_posterior(
    idata,  # InferenceData; uses the posterior group by default
    var_names=["alpha", "beta"],
    ref_val=ref_val_map,
)

In [8]:
%load_ext watermark
%watermark -n -u -v -iv -w -p pymc,pytensor

Last updated: Thu Oct 16 2025

Python implementation: CPython
Python version       : 3.12.11
IPython version      : 9.4.0

pymc    : 5.25.1
pytensor: 2.31.7

pytensor      : 2.31.7
matplotlib    : 3.10.3
pymc          : 5.25.1
pymc_marketing: 0.15.1
xarray        : 2025.7.1
pymc_extras   : 0.4.0
pandas        : 2.3.1
numpy         : 2.2.6
arviz         : 0.22.0

Watermark: 2.5.0

